In [ ]:
import tqdm  # TODO: Agregar un poetry o requirements...
import json
import pandas as pd
import requests

SHEET_ID = '1b6h2RApBs2xbN6-eGVvxH68EALKDklvS91fb7d_IVz4'

# ID de cada sheet
# usuarios -> 102021014
# registros -> 2103959160
# usuarios-all -> 1608673456
# registros-all -> 1249987488
SHEET_TAB_ID = '2103959160'
SHEET_URL = f'https://docs.google.com/spreadsheets/d/{SHEET_ID}/export?format=csv&gid={SHEET_TAB_ID}'

SHEET_URL

'https://docs.google.com/spreadsheets/d/1b6h2RApBs2xbN6-eGVvxH68EALKDklvS91fb7d_IVz4/export?format=csv&gid=2103959160'

In [ ]:
df_registros = pd.read_csv(SHEET_URL)
df_registros.head()

,Timestamp,Padron,Carrera,Mapa
0,9/1/2022 12:22:57,102725,civil,"{""materias"":[{""id"":""CBC28"",""nota"":8},{""id"":""CB..."
1,9/1/2022 12:13:52,105343,informatica,"{""materias"":[{""id"":""CBC28"",""nota"":5},{""id"":""CB..."
2,9/1/2022 11:50:45,105974,informatica,"{""materias"":[{""id"":""CBC28"",""nota"":10},{""id"":""C..."
3,9/1/2022 11:49:45,102889,informatica,"{""materias"":[{""id"":""CBC"",""nota"":0},{""id"":""62.0..."
4,9/1/2022 11:46:00,108215,informatica,"{""materias"":[{""id"":""CBC28"",""nota"":7},{""id"":""CB..."


In [ ]:
print(f"Remuevo los {df_registros[df_registros['Padron'].isnull()]['Timestamp'].count()} registros invalidos")
df_registros = df_registros[df_registros['Padron'].notnull()]
df_registros.head()

Remuevo los 11 registros invalidos


In [ ]:
# TODOs:
# - joinear con "usuarios" para obtener orientacion y tpp/tesis
# - ver si podemos rescatar "checkboxes" del json_mapa
# - ver si podemos migrar el legacy "materia_cuatri" con el nuevo "materia_cuatrimestre"
# - ver por qué hay padrones que no se toman (celda de arriba (teléfono para fede (será tema de formateo de dato numérico en gsheets?)))
dfr = df_registros
df_mapas = None
for idx in tqdm.tqdm(dfr.index):
    json_mapa = json.loads(dfr.loc[idx]['Mapa'])
    df_materias = pd.json_normalize(json_mapa['materias'])
    
    if 'optativas' not in json_mapa:
        df_optativas = pd.DataFrame()
    else:
        df_optativas = pd.json_normalize(json_mapa['optativas'])

    df_materias = df_materias.add_prefix('materia_')
    df_optativas = df_optativas.add_prefix('optativa_')

    df_map = pd.concat([df_materias, df_optativas]).reset_index(drop=True)
    df_map['Padron'] = dfr.loc[idx]['Padron']
    df_map['Carrera'] = dfr.loc[idx]['Carrera']
    df_map['Timestamp'] = dfr.loc[idx]['Timestamp']
    
    if df_mapas is None:
        df_mapas = df_map
    else:
        df_mapas = pd.concat([df_mapas, df_map])

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4230/4230 [00:28<00:00, 149.81it/s]


In [ ]:
df_mapas.head()

,materia_id,materia_nota,materia_cuatrimestre,Padron,Carrera,Timestamp,optativa_id,optativa_nombre,optativa_creditos,materia_cuatri
0,CBC24,5.0,NaN,103779,industrial,8/31/2022 19:57:20,NaN,NaN,NaN,NaN
1,CBC3,7.0,NaN,103779,industrial,8/31/2022 19:57:20,NaN,NaN,NaN,NaN
2,CBC5,7.0,NaN,103779,industrial,8/31/2022 19:57:20,NaN,NaN,NaN,NaN
3,CBC40,9.0,NaN,103779,industrial,8/31/2022 19:57:20,NaN,NaN,NaN,NaN
4,CBC,0.0,NaN,103779,industrial,8/31/2022 19:57:20,NaN,NaN,NaN,NaN


In [ ]:
df_mapas.shape

(94335, 10)

In [ ]:
# Gente que tiene pensado hacer Simu en el futuro...
df_mapas[df_mapas['materia_id'].isin(['75.26']) & (df_mapas['materia_cuatrimestre'] >= 2023)]['Padron']

43    101696
35    104956
41    104393
26     97009
42    107378
32     99248
36    105931
33    106295
40    107964
45    100972
36    102740
39    106004
41    105774
39    106440
47    105970
36    104351
31    108645
35       NaN
40    109071
34    102342
16    109555
40    103784
48    103775
42    103785
39    109131
34    104196
36     23456
Name: Padron, dtype: object